In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, enable=True)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data Pre-processing

### Reading the data converting into TimeSeries format

In [ ]:
def convert_ts_csv_to_df(csvfile):
    '''Take the CSV file along with the path and return dataframe'''
    df = pd.read_csv(csvfile)
    df.insert(loc=0, column='Date', 
                value=pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour', 'Minute']]))
    df.drop(columns=['Year', 'Month', 'Day', 'Hour', 'Minute'], axis=1, inplace=True)
    df.set_index(keys='Date', inplace=True)
    df=df[['Clearsky DHI', 'Clearsky DNI', 'Clearsky GHI', 'Temperature', 'Cloud Type', 'Dew Point', 'Fill Flag', 'Relative Humidity', 'Solar Zenith Angle', 'Pressure', 'Precipitable Water', 'Wind Direction', 'Wind Speed']]
    return df

In [ ]:
df_solar = convert_ts_csv_to_df('data/train.csv')
df_solar.sample(2)

In [ ]:
# Reading the test dataset as Val
# Train dataset shall be split into Train and Test
# Val is read to understand the data and null values
df_solar_test = convert_ts_csv_to_df('data/test_subset_mini.csv')
df_solar_test.sample(2)

### Data Check

##### Null Check

In [ ]:
df_solar.isnull().sum()

In [ ]:
df_solar_test.isnull().sum()

# Feature Selection

##### Datatype Check and convert to floats for Scaling data

In [ ]:
df_solar.info()

In [ ]:
df_solar = df_solar.astype(np.float64)

In [ ]:
df_solar.info()

##### Feature update based on domain

In [ ]:
print('''
Source: https://www.yellowhaze.in/solar-irradiance/
Global Horizontal Irradiance (GHI) = Direct Normal Irradiance (DNI)* cos(solar zenith angle)  +  Diffused Horizontal Irradiance (DHI)''')

In [ ]:
df_solar['Solar Zenith Angle'].describe()

In [ ]:
print('''This shows the Zenith Angle is given in degrees with a min or 35 and max of 171 approximately''')

In [ ]:
from math import cos, radians

In [ ]:
df_solar['Cos Zenith'] = df_solar['Solar Zenith Angle'].apply(lambda x: cos(radians(x)))

In [ ]:
df_solar['Cos Zenith'].describe()

In [ ]:
df_solar.drop(columns='Solar Zenith Angle', inplace=True)

In [ ]:
df_solar.sample(2)

In [ ]:
df_solar.shape

### Adding the Significance of Seasonality into TS
Since we using Deep Learning, the significance of Timestamp is lost and the sinificance behaviour of data with respect to Seasonality. In order to avoid it, these parameters are added

In [ ]:
# Getting the time in seconds
df_solar['ts'] = df_solar.index.map(pd.Timestamp.timestamp)
# Adding columns based on Timestamp to understand Daily and Yearly seasonality
day = 60*60*24
year = 365.2425*day

df_solar['ts_Day_sin'] = np.sin(df_solar['ts'] * (2* np.pi / day))
df_solar['ts_Day_cos'] = np.cos(df_solar['ts'] * (2 * np.pi / day))
df_solar['ts_Year_sin'] = np.sin(df_solar['ts'] * (2 * np.pi / year))
df_solar['ts_Year_cos'] = np.cos(df_solar['ts'] * (2 * np.pi / year))
df_solar.drop(columns='ts', inplace=True)
df_solar.sample(2)

### Scaling of Data

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scalar_std_fs = StandardScaler()

##### Small trick to ensure we can perform inverse transform

In [ ]:
scalar_std_fs = scalar_std_fs.fit(df_solar)

In [ ]:
df_solar.shape

In [ ]:
df_solar_sc = pd.DataFrame(data=scalar_std_fs.transform(df_solar), index=df_solar.index,  columns=df_solar.columns)
df_solar_sc.sample(2)

### Feature Selection

##### Any Non-linear regression model can be selected, Choose XGBoost

In [ ]:
from xgboost import XGBRegressor
from xgboost import plot_importance

##### Feature Selection for Clearsky GHI since GHI constitues both DHI, DHI and Zenith Angle

In [ ]:
y = df_solar_sc['Clearsky GHI']
X = df_solar_sc.drop(columns='Clearsky GHI', axis=1)

In [ ]:
model_xgb = XGBRegressor(n_estimators=100)
model_xgb.fit(X, y)

In [ ]:
model_xgb.feature_importances_

In [ ]:
plot_importance(model_xgb)

### Droping not so significant columns

In [ ]:
df_solar['Cloud Type'].value_counts()

In [ ]:
df_solar.drop(columns=['Cloud Type', 'Pressure', 'Fill Flag', 'Wind Speed', 'Dew Point', 'Wind Direction'], inplace=True)
df_solar.sample(2)

In [ ]:
df_solar.shape

In [ ]:
print('''
Temperature, Relative Humidity, Precipitable Water, Cos of Zenith Angle along with time of Day and time of year components are considered for further analysis
To also reduce the curse of dimensionality'''
)

##### Delete this section - start

### Learning

In [ ]:
from sklearn.preprocessing import StandardScaler
scalar_std_ts = StandardScaler()
a = np.arange(0, 20)
b = np.arange(20, 60)
b.shape
a = a.reshape(4, 5)
a
b = b.reshape(8, 5)
b
scalar_std_ts = scalar_std_ts.fit(b)
b = scalar_std_ts.transform(b)
b.shape
c = b[0:4,:]
scalar_std_ts.inverse_transform(c)
d = b[:, 1:3]
d
scalar_std_ts.inverse_transform(d)

##### Delete this section - end

# Model Building Start

In [49]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, GRU, RepeatVector, Conv1D
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import TimeDistributed

### Consider only last 3 years data

### Data Generation for Time Series

In [4]:
def preprocess_cvs_into_ts(csvfile):
    '''Take the CSV file along with the path and return dataframe'''
    df = pd.read_csv(csvfile)
    df.insert(loc=0, column='Date', 
                value=pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour', 'Minute']]))
    df.drop(columns=['Year', 'Month', 'Day', 'Hour', 'Minute'], axis=1, inplace=True)
    df.set_index(keys='Date', inplace=True)
    df=df[['Clearsky DHI', 'Clearsky DNI', 'Clearsky GHI', 'Temperature', 'Cloud Type', 'Dew Point', 'Fill Flag', 'Relative Humidity', 'Solar Zenith Angle', 'Pressure', 'Precipitable Water', 'Wind Direction', 'Wind Speed']]
    return df

In [83]:
def preprocess_ts_features(df):
    '''This shall ensure the time series features are added and 
    non-significant features are removed'''
    
    # Cos of Zenith angle contributes GHI
    df['Cos Zenith'] = np.cos(np.deg2rad(df['Solar Zenith Angle']))
    
    # Adding the time series features into the dataset for Seasonality
    sec_in_day = 60*60*24
    sec_in_year = 365.2425*sec_in_day
    df['ts'] = df.index.map(pd.Timestamp.timestamp)
    df['ts_Day_sin'] = np.sin(df['ts'] * (2* np.pi / sec_in_day))
    df['ts_Day_cos'] = np.cos(df['ts'] * (2 * np.pi / sec_in_day))
    df['ts_Year_sin'] = np.sin(df['ts'] * (2 * np.pi / sec_in_year))
    df['ts_Year_cos'] = np.cos(df['ts'] * (2 * np.pi / sec_in_year))
    
    # Dropping all the insignificant features
    df.drop(columns=['Cloud Type', 'Pressure', 'Fill Flag', 'Wind Speed', 
                     'Dew Point', 'Wind Direction','ts'], inplace=True)
    
    # Ordering the Columns in a proper order
    df = df[['Clearsky DHI', 'Clearsky DNI', 'Clearsky GHI', 
             'Temperature', 'Relative Humidity', 'Solar Zenith Angle', 
             'Precipitable Water', 'ts_Day_sin', 'ts_Day_cos', 
             'ts_Year_sin', 'ts_Year_cos']]
    return df

In [10]:
df_solar = preprocess_cvs_into_ts('data/train.csv')

In [11]:
df_solar = preprocess_ts_features(df_solar)
df_solar.sample(2)

,Clearsky DHI,Clearsky DNI,Clearsky GHI,Temperature,Relative Humidity,Solar Zenith Angle,Precipitable Water,ts_Day_sin,ts_Day_cos,ts_Year_sin,ts_Year_cos
Date,,,,,,,,,,,
2018-08-01 01:30:00,0,0,0,24.1,100.00,101.75,5.500,0.382683,0.923880,-0.490501,-0.871441
2016-07-27 15:00:00,143,764,741,32.0,55.82,38.48,4.843,-0.707107,-0.707107,-0.430160,-0.902753


In [12]:
from sklearn.preprocessing import StandardScaler
scalar_std_ts = StandardScaler()

In [14]:
scalar_std_ts = scalar_std_ts.fit(df_solar)
solar_data_sc = scalar_std_ts.transform(df_solar)
df_solar_data = pd.DataFrame(data=solar_data_sc, index=df_solar.index, columns=df_solar.columns)
solar_data_sc.shape

(175296, 11)

In [31]:
# Daily there around 48 observations
# Weekly there are around 48*7 = 336 observations
# Monthly there are around 48*30 = 1440 observations
# Yearly there are around 48*365 = 17520 observations
total_data_len = solar_data_sc.shape[0]
batch_input_len = 336
batch_output_len = 1
features_len = 11
train_data_start = total_data_len - batch_input_len*4


In [32]:
def ts_data_generator(data_arr, batch_len):
    X = []
    y = []
    for i in range(len(data_arr)-batch_len):
        X_rows = [row for row in data_arr[i:i+batch_len]]
        X.append(X_rows)
        y_row = [data_arr[i+batch_len][0],data_arr[i+batch_len][1], data_arr[i+batch_len][2]]
        y.append(y_row)
    return np.array(X), np.array(y)

In [33]:
total_data_len, train_data_start, total_data_len-train_data_start, train_split

(175296, 173952, 1344, 1008)

In [34]:
solar_data_sc[train_data_start:].shape

(1344, 11)

In [25]:
X, y = ts_data_generator(solar_data_sc[train_data_start:], batch_input_len)

In [35]:
X.shape, y.shape, train_split

((1008, 336, 11), (1008, 3), 1008)

In [43]:
train_split = np.int(X.shape[0]*0.7)

In [44]:
X[test_split:].shape

(672, 336, 11)

In [45]:
X_train, y_train = X[:train_split], y[:train_split]
X_val, y_val = X[train_split:], y[train_split:]

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((705, 336, 11), (705, 3), (303, 336, 11), (303, 3))

In [47]:
input_shape = (batch_input_len, features_len)

In [50]:
l0 = Input(shape=input_shape)

l1 = GRU(units=8, activation='relu', recurrent_dropout=.2)(l0)
l1 = RepeatVector(input_shape[1])(l1)
l1 = GRU(units=8, activation='relu', recurrent_dropout=.2)(l1)

y = Dense(units=5, activation='relu')(l1)
y = Dense(units=3, activation='linear')(y)

model_lstm1 = Model(inputs=l0, outputs=y)

In [51]:
model_lstm1.compile(optimizer=Adam(learning_rate=0.001), 
                    loss=MeanSquaredError(),
                   metrics=[MeanSquaredError()])
model_lstm1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 336, 11)]         0         
_________________________________________________________________
gru (GRU)                    (None, 8)                 504       
_________________________________________________________________
repeat_vector (RepeatVector) (None, 11, 8)             0         
_________________________________________________________________
gru_1 (GRU)                  (None, 8)                 432       
_________________________________________________________________
dense (Dense)                (None, 5)                 45        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 18        
Total params: 999
Trainable params: 999
Non-trainable params: 0
_______________________________________________________________

In [52]:
# Check points for Early stopping and Saving the best model
cb_earlystop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
cb_modelcp = ModelCheckpoint('best_model_lstm1.h5', monitor='val_mean_squared_error', 
                             mode='min', verbose=1, save_best_only=True)

In [54]:
model_lstm1_hist = model_lstm1.fit(X_train, y_train, 
                                   validation_data=(X_val, y_val), 
                                   epochs=2, callbacks=[cb_earlystop, cb_modelcp])

Epoch 1/2
23/23 [==============================] - 30s 1s/step - loss: 0.7471 - mean_squared_error: 0.7628 - val_loss: 0.6570 - val_mean_squared_error: 0.6583

Epoch 00001: val_mean_squared_error improved from inf to 0.65826, saving model to best_model_lstm1.h5
Epoch 2/2
23/23 [==============================] - 24s 1s/step - loss: 0.6851 - mean_squared_error: 0.6830 - val_loss: 0.6215 - val_mean_squared_error: 0.6235

Epoch 00002: val_mean_squared_error improved from 0.65826 to 0.62348, saving model to best_model_lstm1.h5


In [80]:
y_pred = model_lstm1.predict(X_val[:1:])

In [81]:
y_pred

array([[-0.26638108,  0.06940387,  0.1385091 ]], dtype=float32)

In [66]:
X_val.shape

(303, 336, 11)

In [78]:
X_val[:1:][0][0]

array([ 0.24819735,  1.70509176,  1.0242    , -0.41001228, -0.82661948,
       -0.81175476, -1.48918537, -1.30656296, -0.5411961 , -0.31824423,
        1.37818101])

### Predict

In [127]:
def create_df_predict(train_csv, test_csv, batch_len):
    df_train = preprocess_cvs_into_ts(train_csv)
    df_train = preprocess_ts_features(df_train)
    df_test = preprocess_cvs_into_ts(test_csv)
    df_test = preprocess_ts_features(df_test)
    test_len = df_test.shape[0]
    df_pred = pd.concat([df_train, df_test], axis=0)
    df_pred.fillna(0)
    return df_pred[-(test_len+batch_len):]

In [ ]:
def model_predict(test_csv):
    '''This method takes the test csv file, gives the output as predicted array'''
    df_solar_pred = create_df_predict('data/train.csv', test_csv, batch_input_len)
    solar_data_pred_sc = scalar_std_ts.transform(df_solar_pred)
    X_test, y_test = ts_data_generator(solar_data_pred_sc, batch_input_len)
    
    cnt = 0
    x_pred = []
    for i in range(X_test.shape[0]):
        X_i = np.expand_dims(X_test[i], axis=0)
        print('X_i[-1]\n', X_i[0][-1])
        y_i = model_lstm1.predict(X_i)
        print('y_i\n',y_i)
        for j in range(3):
            X_test[i+1][-1][j] = y_i[0][j]
            print('X_test[i+1][-1][j]\n', X_test[i+1][-1][j], ' y_i[0][j]: ', y_i[0][j])

        print('X_test[i+1][-1]\n', X_test[i+1][-1])
        # print(X_test[i+1][-1][3:])
        x_i_pred = np.hstack([y_i[0], X_test[i+1][-1][3:]])
        print('x_i_pred\n', x_i_pred)
        x_i_inv_sc = scalar_std_ts.inverse_transform(x_i_pred)
        x_pred.append(list(x_i_inv_sc))
        cnt += 1
        if cnt > 5:
            break

    return x_pred
    

In [131]:
df_solar_pred = create_df_predict('data/train.csv', 'data/test.csv', batch_input_len)
df_solar_pred.sample(2)

,Clearsky DHI,Clearsky DNI,Clearsky GHI,Temperature,Relative Humidity,Solar Zenith Angle,Precipitable Water,ts_Day_sin,ts_Day_cos,ts_Year_sin,ts_Year_cos
Date,,,,,,,,,,,
2019-03-15 09:30:00,NaN,NaN,NaN,17.5,97.67,119.34,3.6,6.087614e-01,-0.793353,0.953497,0.301402
2019-05-17 12:00:00,NaN,NaN,NaN,21.4,88.27,74.10,3.5,7.632139e-12,-1.000000,0.711333,-0.702856


In [239]:
cnt = 0
x_pred = []
for i in range(X_test.shape[0]):
    X_i = np.expand_dims(X_test[i], axis=0)
    print('X_i[-1]\n', X_i[0][-1])
    y_i = model_lstm1.predict(X_i)
    print('y_i\n',y_i)
    for j in range(3):
        X_test[i+1][-1][j] = y_i[0][j]
        print('X_test[i+1][-1][j]\n', X_test[i+1][-1][j], ' y_i[0][j]: ', y_i[0][j])
    
    print('X_test[i+1][-1]\n', X_test[i+1][-1])
    # print(X_test[i+1][-1][3:])
    x_i_pred = np.hstack([y_i[0], X_test[i+1][-1][3:]])
    print('x_i_pred\n', x_i_pred)
    x_i_inv_sc = scalar_std_ts.inverse_transform(x_i_pred)
    x_pred.append(list(x_i_inv_sc))
    cnt += 1
    if cnt > 5:
        break

print(x_pred)

X_i[-1]
 [-6.59968704e-04 -3.46991196e-02 -8.28368366e-02  1.04352437e-01
  8.87461866e-01  2.61705099e-01  5.14725668e-01 -1.84591911e-01
  1.40211477e+00  2.35251224e-03  1.41445850e+00]
y_i
 [[-0.01607201 -0.01058526 -0.07993868]]
X_test[i+1][-1][j]
 -0.01607201062142849  y_i[0][j]:  -0.01607201
X_test[i+1][-1][j]
 -0.010585262440145016  y_i[0][j]:  -0.010585262
X_test[i+1][-1][j]
 -0.07993867993354797  y_i[0][j]:  -0.07993868
X_test[i+1][-1]
 [-1.60720106e-02 -1.05852624e-02 -7.99386799e-02  5.96250708e-02
  9.31165096e-01  4.11635235e-01  3.85441085e-01 -7.11253528e-12
  1.41421356e+00  2.85932357e-03  1.41445757e+00]
x_i_pred
 [-1.60720106e-02 -1.05852624e-02 -7.99386799e-02  5.96250708e-02
  9.31165096e-01  4.11635235e-01  3.85441085e-01 -7.11253528e-12
  1.41421356e+00  2.85932357e-03  1.41445757e+00]
X_i[-1]
 [-1.60720106e-02 -1.05852624e-02 -7.99386799e-02  5.96250708e-02
  9.31165096e-01  4.11635235e-01  3.85441085e-01 -7.11253528e-12
  1.41421356e+00  2.85932357e-03  1.4144

### Scaling the Updated Data

In [ ]:
scalar_std = StandardScaler()
scalar_std = scalar_std.fit(df_solar)
df_solar_sc = pd.DataFrame(data=scalar_std.transform(df_solar), index=df_solar.index,  columns=df_solar.columns)
df_solar_sc.sample(2)

### Data Generation

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
# # Take only one weeks data for completing the process
# # total_length = 17520*2
# total_length = df_solar_sc.shape[0]

### Model Building

In [ ]:
model_lstm1.load_weights('best_model_lstm1.h5')

In [ ]:
def plot_model_history(model):
    mod_hist = model.history
    plt.subplot(1, 2, 1)
    plt.plot(mod_hist.history['loss'])
    plt.plot(mod_hist.history['val_loss'])
    plt.title('Loss MSE')
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.subplot(1, 2, 2)
    plt.plot(mod_hist.history['root_mean_squared_error'])
    plt.plot(mod_hist.history['val_root_mean_squared_error'])
    plt.title('Error RMSE')
    plt.ylabel('Loss')
    plt.xlabel('Epochs')

In [ ]:
plot_model_history(model=model_lstm1)

##### Inverse Scaling the data

In [ ]:
plt.figure(figsize=(12, 9))
cnt = 1
for col in df_solar_pred.columns:
    plt.subplot(3, 1, cnt)
    plt.plot(df_solar_train[col], label='train')
    plt.plot(df_solar_test[col], label='test')
    plt.plot(df_solar_pred[col], label='Pred')
    plt.xticks(rotation=45)
    plt.legend(loc='best')
    cnt += 1

plt.tight_layout()
plt.show()

### Predicting for Future (Forecasting)